In [1]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F

class fn_cls(nn.Module):
    def __init__(self,device):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-uncased")
        self.model.resize_token_embeddings(len(tokenizer))##############
        self.model.to(device)
#         self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, leishu)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
#         print(outputs[0])torch.Size([8, 100, 768])
#         print(outputs[1])torch.Size([8, 768])
#         print(outputs[0][:,0,:])torch.Size([8, 768])
        x = outputs[1]
#         x = self.dropout(x)
        x = self.l1(x)
        return x


In [2]:
import torch
device0 = torch.device('cuda:3' if torch.cuda.is_available() else "cpu")#训练集gpu
cls=torch.load("../data/cls_0.88785_266.58456.model",map_location=device0)

In [3]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
softmax = nn.Softmax(dim=1)
def predict(device,s_l,cls):
    with torch.no_grad():
        cls.to(device)
        cls.eval()
        text2id = tokenizer(
            s_l, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
        )
        input_ids=text2id["input_ids"].to(device)
        mask=text2id["attention_mask"].to(device)
        output = cls(input_ids, attention_mask=mask)

        output1=softmax(output)
        output2=output.argmax(dim=1)
        return output1,output2

In [4]:
s=['Echoes Repeats Success','"Stocks Finish Lower, Retail Sector Weighs"','Report indicates Wannstedt out','Conference Members Back Iraqi Efforts']
print(predict(device0,s,cls))

(tensor([[1.6244e-02, 9.7378e-01, 5.3007e-03, 4.6791e-03],
        [6.1406e-02, 1.4047e-03, 9.3121e-01, 5.9830e-03],
        [7.6313e-03, 9.8954e-01, 2.2822e-03, 5.4985e-04],
        [9.9728e-01, 8.1377e-04, 1.0767e-03, 8.2985e-04]], device='cuda:3'), tensor([1, 2, 1, 0], device='cuda:3'))


In [5]:
import pandas as pd
def text2token(text,tokenizer,max_length=100):
    text2id = tokenizer(
        text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
    )
    input_ids=text2id["input_ids"].tolist()
    attention_mask=text2id["attention_mask"].tolist()
    return input_ids,attention_mask
def data2token(data_,tokenizer):
    text=[i for i in data_['title'].values]
    input_ids,attention_mask=text2token(text,tokenizer)
    data_['input_ids']=input_ids
    data_['attention_mask']=attention_mask
    return data_
data_test1=pd.read_csv("../data/data_test1.csv")
data_test1=data2token(data_test1,tokenizer)

In [21]:
from torch.utils.data import Dataset
class SentimentDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "title"]
        label = self.dataset.loc[idx, "label"]
        input_ids = self.dataset.loc[idx, "input_ids"]
        attention_mask = self.dataset.loc[idx, "attention_mask"]
        sample = {"text": text, "label": label,"input_ids":input_ids,"attention_mask":attention_mask}
        # print(sample)
        return sample
    
#按batch_size分
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import torch

batch_size=16
test1_loader = DataLoader(
    SentimentDataset(data_test1), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

In [22]:
from tqdm import tqdm
from sklearn import metrics
def predict_loader(device,test_loader,cls):
    with torch.no_grad():
        cls.to(device)
        cls.eval()
        output_all=[]
        label_all=[]
        for batch_idx,batch in enumerate(test_loader):
            print(str(batch_idx)+'/'+str(len(test_loader)),end='\r')
            label=batch['label'].to(device)#batch size * 1
            label_all.append(label.view(-1,1))
            input_ids=torch.stack(batch['input_ids']).t().to(device)#batch size * 100
            attention_mask=torch.stack(batch['attention_mask']).t().to(device)#batch size * 100
            
            #计算输出
            output = cls(input_ids, attention_mask=attention_mask)#batch size * 1
            
            #四舍五入
            output=softmax(output)
            output=output.argmax(dim=1)
            output_all.append(output)
        output_all=torch.cat(output_all,0)
        label_all=torch.cat(label_all,0)

        output_all=np.array(output_all.cpu())
        label_all=np.array(label_all.cpu())
        acc_score=metrics.accuracy_score(label_all,output_all)
        print("准确率:"+str(acc_score))
        print(metrics.classification_report(label_all,output_all))
        return label_all,output_all
        

In [23]:
label_all,output_all=predict_loader(device0,test1_loader,cls)

准确率:0.8880485893416928
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      6380
           1       0.92      0.95      0.93      6380
           2       0.85      0.85      0.85      6380
           3       0.87      0.87      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [24]:
label_all

array([[3],
       [0],
       [2],
       ...,
       [2],
       [2],
       [1]])

In [25]:
data_test1

,label,title,input_ids,attention_mask
0,3,Salesforce.com Launches On-Demand Support System,"[101, 4341, 14821, 1012, 4012, 18989, 2006, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
1,0,W: Kerry an in-fidel,"[101, 1059, 1024, 11260, 2019, 1999, 1011, 260...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
2,2,Rate of oil exploration slows,"[101, 3446, 1997, 3514, 8993, 4030, 2015, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,2,US interest rate rise expected,"[101, 2149, 3037, 3446, 4125, 3517, 102, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,3,Echoes Repeats Success,"[101, 17659, 17993, 3112, 102, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
25515,0,Files reveal 1953 Gibraltar row,"[101, 6764, 7487, 4052, 12272, 5216, 102, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
25516,0,Denmark mourns its tsunami dead,"[101, 5842, 9587, 14287, 2015, 2049, 19267, 27...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
25517,2,Chavez #39;s victory,"[101, 16860, 1001, 4464, 1025, 1055, 3377, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
25518,2,ViewPoints: Unfriendly skies,"[101, 21386, 2015, 1024, 4895, 19699, 9013, 18...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


In [26]:
data_test1['pre']=output_all
data_test1

,label,title,input_ids,attention_mask,pre
0,3,Salesforce.com Launches On-Demand Support System,"[101, 4341, 14821, 1012, 4012, 18989, 2006, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",3
1,0,W: Kerry an in-fidel,"[101, 1059, 1024, 11260, 2019, 1999, 1011, 260...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",0
2,2,Rate of oil exploration slows,"[101, 3446, 1997, 3514, 8993, 4030, 2015, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",2
3,2,US interest rate rise expected,"[101, 2149, 3037, 3446, 4125, 3517, 102, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
4,3,Echoes Repeats Success,"[101, 17659, 17993, 3112, 102, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
...,...,...,...,...,...
25515,0,Files reveal 1953 Gibraltar row,"[101, 6764, 7487, 4052, 12272, 5216, 102, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
25516,0,Denmark mourns its tsunami dead,"[101, 5842, 9587, 14287, 2015, 2049, 19267, 27...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...",0
25517,2,Chavez #39;s victory,"[101, 16860, 1001, 4464, 1025, 1055, 3377, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",0
25518,2,ViewPoints: Unfriendly skies,"[101, 21386, 2015, 1024, 4895, 19699, 9013, 18...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",3


In [32]:
data_test1.to_csv("../data/data_test1_out.csv",index=0)

In [29]:
data_test1_bad=data_test1.loc[data_test1['label']!=data_test1['pre']]

In [33]:
data_test1_bad.to_csv("../data/data_test1_bad.csv",index=0)

In [34]:
del data_test1_bad['input_ids']
del data_test1_bad['attention_mask']
del data_test1_bad['pre']



In [35]:
data_test1_bad

,label,title
4,3,Echoes Repeats Success
5,0,Weak Demand Leads Google to Lower Its Sights
14,0,Lost city of Atlantis found?
17,2,Novell Files Another Suit Against Microsoft
37,0,Brother may #39;buy #39; Bigley
...,...,...
25476,0,Future of EU Patent Legislation in Doubt (AP)
25491,3,PeopleSoft #39;s Prospects Top Agenda at Show
25514,1,Tragedy touches a life near-perfect
25517,2,Chavez #39;s victory


In [36]:
data_test1_bad0=data_test1_bad.loc[data_test1_bad['label']==0]
data_test1_bad1=data_test1_bad.loc[data_test1_bad['label']==1]
data_test1_bad2=data_test1_bad.loc[data_test1_bad['label']==2]
data_test1_bad3=data_test1_bad.loc[data_test1_bad['label']==3]
data_test1_bad0

,label,title
5,0,Weak Demand Leads Google to Lower Its Sights
14,0,Lost city of Atlantis found?
37,0,Brother may #39;buy #39; Bigley
46,0,Clinton Library Reflects Its Subject's Volatil...
108,0,Barroso opts for last minute #39;face saver #39;
...,...,...
25333,0,Fall from Olympus: Athens to Beijing
25338,0,Senate Hears Testimony on Porn Addiction (AP)
25348,0,Stocks Higher Despite Soaring Oil Prices
25410,0,Stocks Are Up Despite Rising Oil Prices


In [38]:
data_train=pd.read_csv("../data/data_train.csv")
data_val=pd.read_csv("../data/data_val.csv")

In [40]:
def pin(data_train,data_val,bad):
    len_=len(bad)
    data_train=pd.concat([data_train,bad[0:int(len_*5/6)]])
    data_val=pd.concat([data_val,bad[int(len_*5/6):len_]])
    return data_train,data_val
    
    
    

In [41]:
print(data_train.shape,data_val.shape)
data_train,data_val=pin(data_train,data_val,data_test1_bad0)
print(data_train.shape,data_val.shape)
data_train,data_val=pin(data_train,data_val,data_test1_bad1)
print(data_train.shape,data_val.shape)
data_train,data_val=pin(data_train,data_val,data_test1_bad2)
print(data_train.shape,data_val.shape)
data_train,data_val=pin(data_train,data_val,data_test1_bad3)
print(data_train.shape,data_val.shape)

(63800, 2) (12760, 2)
(64425, 2) (12886, 2)
(64710, 2) (12943, 2)
(65507, 2) (13103, 2)
(66179, 2) (13238, 2)


In [43]:
data_train.to_csv("../data/data_train2.csv",index=0)
data_val.to_csv("../data/data_val2.csv",index=0)